This notebook experiments with building out a fully customized TerriaJS catalog from a search result of ScienceBase Items, transforming sbJSON into the json catalog format used by TerriaJS. Resulting catalog JSON files are output to the local directory, deployed to GitHub, and then callable into an out of the box TerriaJS mapper.

In [6]:
import requests,json
from IPython.display import display

In [7]:
# Get what we want to work with using a ScienceBase /items/ query
sbURL = "https://www.sciencebase.gov/catalog/items?q=&max=20&filter=browseType%3DArcGIS%20Service%20Definition&folderId=4f4e4773e4b07f02db47e241&format=json&fields=title,body,distributionLinks"
sbJSON = requests.get(sbURL).json()



In [24]:
theCatalog = {}
theCatalog["corsDomains"] = ["sciencebase.gov","corsproxy.com"]
theCatalog["baseMapName"] = "Bing Maps Roads"
theCatalog["initialViewerMode"] = "2d"
theCatalog["services"] = []

theCatalog["homeCamera"] = {}
theCatalog["homeCamera"]["west"] = -175
theCatalog["homeCamera"]["east"] = -135
theCatalog["homeCamera"]["north"] = 72
theCatalog["homeCamera"]["south"] = 55

theCatalog["catalog"] = []

nfhpGroup = {}
nfhpGroup["name"] = "NFHP 2010"
nfhpGroup["type"] = "group"
nfhpGroup["isOpen"] = "true"

nfhpGroup["items"] = []

for idx, item in sbJSON["items"]:
    thisItem = {}
    thisItem["name"] = item["title"]
    serviceCount = 0

    if idx < 4:
        thisItem["isEnabled"] = "true"

    for link in item["distributionLinks"]:
        if link["title"] == "ArcGIS REST Service":
            serviceCount = serviceCount + 1
            if serviceCount > 1:
                break
            thisItem["type"] = "esri-mapServer-group"
            thisItem["url"] = link["uri"]
    nfhpGroup["items"].append(thisItem.copy())

theCatalog["catalog"].append(nfhpGroup.copy())

f = open('catalog.json', 'wt', encoding='utf-8')
f.write(json.dumps(theCatalog).replace(': "true"', ': true'))

ValueError: too many values to unpack (expected 2)